# Итераторы


In [150]:
iterator = iter([1, 2, 3])

In [155]:
# print(next(iterator))

In [41]:
class EvenIterator:
    def __init__(self, start, end):
        self.current = start
        self.end = end
        
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.current > self.end:
            raise StopIteration

        self.current += 2
        return self.current - 2
    
    
# for num in EvenIterator(1, 10):
#     print(num)
        

In [37]:
class IndexIterable:
    def __init__(self, obj):
        self.obj = obj
        
    def __getitem__(self, index):
        return self.obj[index]


# for letter in IndexIterable('123'):
#     print(letter)


# Какой еще объект может быть вместо строки?

In [34]:
import collections



# isinstance(EvenIterator(1, 10), collections.Iterable)
# isinstance(IndexIterable('123'), collections.Iterable)


# Асинхронные итераторы

In [91]:
import asyncio


class Ranger:
    def __init__(self, limit):
        self.limit = limit

    async def __aiter__(self):
        return self

    async def __anext__(self):
        await asyncio.sleep(1)
        if self.limit == 0:
            raise StopAsyncIteration
            
        self.limit -= 1
        return self.limit + 1
       

async def main():
    async for val in Ranger(10):
        print(val)

    
# loop = asyncio.get_event_loop()
# loop.run_until_complete(main())

# Генераторы

### Что такое генераторы и зачем они нужны?

In [40]:
def fibonacci(number):
    a = b = 1
    for _ in range(number):
        yield a
        a, b = b, a + b
        

# for num in fibonacci(100):
#     print(num)

In [162]:
def list_generator(list_obj):
    for item in list_obj:
        yield item
        print('After yielding {}'.format(item))


generator = list_generator([1, 2, 3])

In [168]:
# print(next(generator))

In [113]:
def accumulator():
    total = 0
    while True:
        value = yield total
        if not value:
            break

        total += value
        

g = accumulator()

# next(g)

# print(g.send(1))
# print(g.send(2))
# print(g.send(3))

# next(g)

In [123]:
def catcher():
    try:
        while True:
            yield 1
    except ValueError:
        yield 2
    

g = catcher()

# next(g)

# g.throw(ValueError)

# next(g)

# Асинхронные генераторы

### Планируются в 3.6

# Списочные выражения

In [124]:
import random


def get_random_list(length):
    random_list = []
    for _ in range(length):
        random_list.append(random.random())
    
    return random_list


# print(get_random_list(10))

In [126]:
import random


def get_random_list(length):
    return [random.random()] * length

# print(get_random_list(10))

In [128]:
import random


def get_random_list(length):
    return [random.random() for _ in range(length)]

# print(get_random_list(10))

In [115]:
# [str(num) for num in range(10)]

In [117]:
# [str(num) for num in range(10) if num % 2]

In [102]:
# [str(num) if num % 2 else num for num in range(10)]

In [132]:
# {num % 10 for num in range(100)}

In [136]:
# {num: num % 3 for num in range(20)}

In [ ]:
# dict((x, x ** 2) for x in range(4))

In [138]:
# (num ** 2 for num in range(10))

In [14]:
# [i * j for i in range(5) for j in range(5, 10)]

In [17]:
# [[x for x in range(5)] for _ in range(5)]

In [99]:
# [x.sort() for x in [[2, 1], [4, 3], [0, 1]]]

In [ ]:
# [print(x) for x in (1, 2, 3)]

In [139]:
a = [x ** 2 for x in range(1, 5)]
b = [x ** 3 for x in range(1, 5)]

# list(zip(a, b))

In [138]:
# list(zip(a * 2, b))

[(1, 1), (4, 8), (9, 27), (16, 64)]

# Контекстные менеджеры

In [55]:
with open('access_log.log', 'a') as f:
    f.write('New Access\n')

In [78]:
class open_file:
    def __init__(self, filename, mode):
        self.f = open(filename, mode)
    
    def __enter__(self):
        return self.f
    
    def __exit__(self, *args):
        self.f.close()
        
        
# with open_file('test.log', 'a') as f:
#     f.write('lalala\n')
    
# with open_file('test.log', 'r') as f:
#     print(f.readlines())

In [87]:
class suppress_exception:
    def __init__(self, exc_type):
        self.exc_type = exc_type
    
    def __enter__(self):
        return self
    
    def __exit__(self, exc_type, exc_value, traceback):
        if exc_type == self.exc_type:
            print('Nothing happend.')
            return True
    
    
# with suppress_exception(ZeroDivisionError) as f:
#     print(type(f).__name__)
#     big_number = 1 / 0


In [116]:
import contextlib


@contextlib.contextmanager
def printer(num):
    print('Enter')
    
    yield num ** 3
    
    print('Exit')

    
# with printer(2) as p:
#     print(p)

## Напишите контекстный менеджер который считает и выводит время, проведенное внутри него

In [103]:
import random
import time


with timer():
    for _ in range(10):
        time.sleep(random.random())

Elapsed time: 5.103878974914551


## Как добавить возможность получить текущее пройденное время?

In [105]:
import random
import time


with timer() as t:
    for _ in range(10):
        time.sleep(random.random())
        print(t.elapsed)

0.5866799354553223
0.7167010307312012
0.8294858932495117
0.8603990077972412
1.756950855255127
1.9732298851013184
2.197551965713501
3.140017032623291
3.8523449897766113
4.186240911483765
Elapsed time: 4.1863789558410645


# Асинхронные контекстные менеджеры

In [49]:
import aiohttp
import asyncio


async def fetch(client):
    async with client.get('http://python.org') as resp:
        assert resp.status == 200
        return await resp.text()


async def main(loop):
    async with aiohttp.ClientSession(loop=loop) as client:
        html = await fetch(client)
        print(html[:96])

        
loop = asyncio.get_event_loop()
# loop.run_until_complete(main(loop))

# Дескрипторы

In [21]:
class Descriptor:
    def __get__(self, obj, obj_type):
        print('get')
        
    def __set__(self, obj, value):
        print('set')


class Class:
    attr = Descriptor()
    

instance = Class()

# instance.attr
# instance.attr = 10

In [44]:
class Value:
    def __init__(self):
        self.value = None
    
    @staticmethod
    def _prepare_value(value):
        return value * 10

    def __get__(self, obj, obj_type):
        return self.value
    
    def __set__(self, obj, value):
        self.value = self._prepare_value(value)
    
    
class Class:
    attr = Value()

    
instance = Class()
instance.attr = 10

# print(instance.attr)
# print(Class.attr)

In [19]:
# Что происходит в момент доступа к атрибуту — instance.attr (Чтение)

def pseudo_read():
    if hasattr(instance.__class__, 'attr'):
        obj = instance.__class__.attr
        obj_type = obj.__class__

        if hasattr(obj_type, '__get__') and (
            hasattr(obj_type, '__set__') or 'attr' not in instance.__dict__
        ):
            return obj_type.__get__(obj, instance, instance.__class__)

    return instance.__dict__['attr']   


# 1. Data Descriptors
# 2. instance.__dict__
# 3. Non-Data Descriptors
# 4. class.__dict__
# 5. base classes


# instance.attr => type(instance).__dict__['attr'].__get__(instance, type(instance))
# Class.attr => Class.__dict__['attr'].__get__(None, Class)
    

In [25]:
# Что происходит во время — instance.attr = 10 (Записи)

def pseudo_write():
    if hasattr(instance.__class__, 'attr'):
        obj = instance.__class__.attr
        obj_type = obj.__class__

        if hasattr(obj_type, '__set__'):
            obj_type.__set__(obj, instance, 10)
            return

    instance.__dict__['attr'] = 10


## Напишите дескриптор, который пишет в переданный ему во время инициализации файл все присваиваемые значения

In [ ]:
class PersistentField:
    pass


class Class:
    attr = PersistentField(filename='fast.db')

### Функции — это дескрипторы

In [52]:
class Class:
    def method(self):
        pass
    
    
obj = Class()    
print(obj.method)
print(Class.method)

<bound method Class.method of <__main__.Class object at 0x107be5390>>
<function Class.method at 0x107c4d620>


In [70]:
def summator(x, y):
    return x + y


lolwhat = summator.__get__(10)
# lolwhat(21)

### @classmethod & @staticmethod & @property — это дескрпиторы

In [32]:
class Property:
    def __init__(self, getter):
        self.getter = getter

    def __get__(self, obj, objtype=None):
        if obj is None:
            return self
    
        return self.getter(obj)
    
    
class Class:
    @property
    def x(self):
        return 'x'
    
    @Property
    def y(self):
        return 'y'
    
    def get_z(self):
        return 'z'
    
    z = Property(get_z)
    

obj = Class()

# print(obj.x)
# print(obj.y)
# print(obj.z)

## Напишите реализацию @staticmethod или @classmethod

In [43]:
   
class Class:
    @ClassMethod
    def test(cls):
        print(cls.__name__)
    

a = Class()
a.test()


Class


In [50]:
class Class:
    @StaticMethod
    def test():
        print('Look Mum No Hands!')
        

obj = Class()
obj.test()

Look Mum No Hands!


# \_\_slots\_\_ (и здесь не без дескрипторов)

In [69]:
class Class:
    __slots__ = ['hello']
    
    def __init__(self):
        self.test = 'test'

        
# obj = Class()

# Мета-классы

In [129]:
class A:
    ...


In [133]:
a = A()

# print(type(a))
# print(type(A))

# print(isinstance(a, A))
# print(isinstance(A, type))

# print(issubclass(A, type))
# print(issubclass(A, object))

In [98]:
def dummy_factory():
    class Class:
        pass
    
    return Class


Dummy = dummy_factory()
obj = Dummy()

In [10]:
class Meta(type):
    pass


class A(metaclass=Meta):
    pass


# print(type(A))

In [57]:
from abc import ABC, abstractmethod


class Abstract(ABC):
    @abstractmethod
    def abstract_method(self):
        pass


# Abstract()

# Несколько полезных модулей стандартной библиотеки

### itertools

In [2]:
import itertools

In [24]:
# list(itertools.chain(range(10), range(10, 20)))

In [13]:
# list(itertools.combinations(range(5), 3))

In [ ]:
# list(itertools.permutations(range(3), 3))

In [5]:
# g = itertools.cycle('ABC')
# for _ in range(10):
#     print(next(g))

In [8]:
for number in itertools.count():
    if number > 10:
        break
    
#    print(number)

In [22]:
# list(itertools.repeat(1, 10))

In [16]:
# list(itertools.product('ABC', 'xy'))

In [20]:
# list(itertools.zip_longest(range(10), range(5), fillvalue=None))

### re

In [28]:
import re

In [134]:
sentence = 'Call me ASAP!!!11 8-990-1122239 srsly.'
# re.search(r' ([\d-]+) ', sentence).groups()

In [121]:
# re.findall(r"(\w+)", "Python is the capital of Great Britain")

In [120]:
# re.search(r"(123)", "a123zzb").group(1)

In [131]:
# re.sub('(\.)', ' :)', 'Hello...')

In [30]:
# ints = re.compile(r"(\d+)")
# re.findall(r"(123)", "a123zaa223zb")

### collections

In [31]:
import collections

In [40]:
colors = ['red', 'blue', 'red', 'green', 'blue', 'blue']
counter = collections.Counter(colors)

# print(counter)

In [43]:
# print(counter.most_common(2))

In [88]:
d = collections.defaultdict(lambda: 'not here')   
# print(d['missing_key'])

In [89]:
# collections.OrderedDict()

In [69]:
class SuperList(collections.abc.MutableSequence):
    pass


### datetime

In [91]:
from datetime import datetime, timedelta

In [108]:
midterm = datetime.now() + timedelta(days=7)
# print(midterm.day)

In [101]:
# print(datetime.now().strftime('%Y / %m / %d'))

In [104]:
# print(datetime.strptime('2016-10-25', '%Y-%m-%d'))